## **Using sklearn to preprocessing data and tensorflow for modeling**
***Preprocessing:***
* Focus only in feature scaling (normalise) and feature combine (PCA)
* Building pipeline

***Modeling:***
* MLP
inspired by https://www.kaggle.com/riadalmadani/pytorch-cv-0-0145-lb-0-01839#Single-fold-training

In [24]:
import numpy as np
import pandas as pd
import copy

import tensorflow as tf
from tensorflow import feature_column as fc
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.layers import Dense, DenseFeatures, Dropout, BatchNormalization, Embedding, Input, Concatenate, Average
from tensorflow.keras.metrics import AUC, Precision, Recall
from tensorflow.keras import backend as K, Sequential, Model
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping

from tensorflow_addons.layers import WeightNormalization
from keras.wrappers.scikit_learn import KerasRegressor

from sklearn.preprocessing import QuantileTransformer, StandardScaler
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.compose import ColumnTransformer
from sklearn.feature_selection import VarianceThreshold
from sklearn.decomposition import PCA

import matplotlib.pyplot as plt
from math import log2

print(pd.__version__)
print(tf.__version__)

1.1.3
2.3.1


## Data loading

In [2]:
# Loading data and encoding

folder_path = '../input/lish-moa/'
raw_test = pd.read_csv(folder_path + 'test_features.csv')
raw_train = pd.read_csv(folder_path + 'train_features.csv')
raw_targets = pd.read_csv(folder_path + 'train_targets_scored.csv')

# Phân loại dữ liệu
cols_id = ['sig_id']
cols_to_remove = ['cp_type']
cols_fts = [i for i in raw_train.columns if i not in cols_id +cols_to_remove]
cols_gene = [col for col in raw_train.columns if col.startswith("g-")]
cols_cell = [col for col in raw_train.columns if col.startswith("c-")]
cols_experiment = [col for col in cols_fts if col not in cols_gene+cols_cell]
cols_target = [i for i in raw_targets.columns if i not in cols_id]
num_fts, num_labels = len(cols_fts), len(cols_target)

# xử lý categorical
def transform_data(input_data):
    '''Clean data and encoding
        * input_data: table '''
    out = input_data.copy()
    out['cp_dose'] = out['cp_dose'].map({'D1':0, 'D2':1})
    out['cp_time'] = out['cp_time']/72
    
    return out

to_train = transform_data(raw_train[raw_train['cp_type'] != 'ctl_vehicle'])
to_train_targets = raw_targets.iloc[to_train.index]
to_pred  = transform_data(raw_test)
to_pred_non_ctl = to_pred[to_pred['cp_type'] != 'ctl_vehicle']

## Feature engineering

In [50]:
# preprocessing pipeline
def pipe_line_builder( quantiles_num, pca_dims):
    '''Dựng pipe line cho từng nhóm columns
    :quantiles_num: int: số quantile khi normalise
    :pca_dims: int: số chiều pca'''
#     variance = VarianceThreshold(variance_threshold)
    norm = StandardScaler()
    #QuantileTransformer(n_quantiles=quantiles_num,random_state=0, output_distribution="normal")
    pca = PCA(n_components = pca_dims)
    p_var_norm = Pipeline([ 
#         ('var', variance),
        ('norm', norm) ])
    p_var_norm_pca = Pipeline([ 
#         ('var', variance),
        ('pca', pca),
        ('norm', norm)
    ])
    return FeatureUnion([
        ('norm', p_var_norm), 
        ('norm_pca', p_var_norm_pca) ])

pipe = ColumnTransformer([
     ('gene', pipe_line_builder(quantiles_num = 100, pca_dims = 640), cols_gene),
     ('cell', pipe_line_builder(quantiles_num = 100, pca_dims = 72), cols_cell),
     ('experiment', pipe_line_builder(quantiles_num = 100, pca_dims = int(len(cols_experiment)*0.8)), cols_experiment)
#      ('all', Pipeline([ 
#                 ('norm', QuantileTransformer(n_quantiles=100,random_state=0, output_distribution="normal")), 
#                 ('pca', PCA(n_components = int(len(cols_fts)*0.8)) ) ]), cols_fts)
    ])

In [51]:
# Final data
# Trick
pipe.fit(to_train[cols_fts].append(to_pred[cols_fts]))
X_train = pipe.transform(to_train[cols_fts])
y_train = to_train_targets[cols_target]

In [52]:
X_train.shape

(21948, 1587)

In [41]:
len(cols_gene+cols_cell+cols_experiment)+640+72+int(len(cols_experiment)*0.8)

1587

## Modeling

In [53]:
# Define model
model = Sequential([
    BatchNormalization(),
    WeightNormalization(Dense(1024, activation="relu")),
    BatchNormalization(),
    Dropout(0.2),
    WeightNormalization(Dense(512, activation="relu")),
    BatchNormalization(),
    Dropout(0.2),
    WeightNormalization(Dense(256, activation="relu")),
    BatchNormalization(),
    Dropout(0.2),
    WeightNormalization(Dense(num_labels, activation="sigmoid"))
])

p_min = 0.001
p_max = 0.999
def logloss(y_true, y_pred):
    y_pred = tf.clip_by_value(y_pred,p_min,p_max)
    return -K.mean(y_true*K.log(y_pred) + (1-y_true)*K.log(1-y_pred))

model.compile(optimizer='adam', loss=BinaryCrossentropy(label_smoothing=0.0001), metrics=logloss)

In [54]:
# Training and evaluate
reduce_lr = ReduceLROnPlateau(monitor='val_logloss', factor=0.3, patience=5, mode='min', min_lr=1E-5, verbose=1)
early_stopping = EarlyStopping(monitor='val_logloss', min_delta=1E-5, patience=15, mode='min',restore_best_weights=True, verbose=1)
# reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.3, patience=3,epsilon = 1e-4, mode = 'min',verbose=1)
# early_stopping = EarlyStopping(monitor='val_loss',min_delta=0,patience=10,mode='auto',verbose=1,baseline=None,restore_best_weights=True)

hist = model.fit(X_train,y_train, batch_size=64, epochs=150,validation_split = 0.2
                 ,callbacks=[reduce_lr, early_stopping]
                )

Epoch 1/150
275/275 [==============================] - 3s 10ms/step - loss: 0.2852 - logloss: 0.2845 - val_loss: 0.0354 - val_logloss: 0.0352
Epoch 2/150
275/275 [==============================] - 3s 10ms/step - loss: 0.0275 - logloss: 0.0272 - val_loss: 0.0222 - val_logloss: 0.0219
Epoch 3/150
275/275 [==============================] - 2s 7ms/step - loss: 0.0216 - logloss: 0.0213 - val_loss: 0.0201 - val_logloss: 0.0199
Epoch 4/150
275/275 [==============================] - 2s 9ms/step - loss: 0.0197 - logloss: 0.0194 - val_loss: 0.0189 - val_logloss: 0.0187
Epoch 5/150
275/275 [==============================] - 2s 7ms/step - loss: 0.0186 - logloss: 0.0183 - val_loss: 0.0184 - val_logloss: 0.0181
Epoch 6/150
275/275 [==============================] - 2s 9ms/step - loss: 0.0177 - logloss: 0.0174 - val_loss: 0.0179 - val_logloss: 0.0177
Epoch 7/150
275/275 [==============================] - 2s 9ms/step - loss: 0.0169 - logloss: 0.0167 - val_loss: 0.0179 - val_logloss: 0.0176
Epoch 8/150

In [57]:
# Test thử resnet
p_min = 0.001
p_max = 0.999
def logloss(y_true, y_pred):
    y_pred = tf.clip_by_value(y_pred,p_min,p_max)
    return -K.mean(y_true*K.log(y_pred) + (1-y_true)*K.log(1-y_pred))

def build_model(n_features, n_features_2, n_labels, label_smoothing = 0.0005):    
    input_1 = Input(shape = (n_features,), name = 'Input1')
    input_2 = Input(shape = (n_features_2,), name = 'Input2')

    head_1 = Sequential([
        BatchNormalization(),
        Dropout(0.2),
        Dense(512, activation="elu"), 
        BatchNormalization(),
        Dense(256, activation = "elu")
        ],name='Head1') 

    input_3 = head_1(input_1)
    input_3_concat = Concatenate()([input_2, input_3])

    head_2 = Sequential([
        BatchNormalization(),
        Dropout(0.3),
        Dense(512, "relu"),
        BatchNormalization(),
        Dense(512, "elu"),
        BatchNormalization(),
        Dense(256, "relu"),
        BatchNormalization(),
        Dense(256, "elu")
        ],name='Head2')

    input_4 = head_2(input_3_concat)
    input_4_avg = Average()([input_3, input_4]) 

    head_3 = Sequential([
        BatchNormalization(),
        Dense(256, kernel_initializer='lecun_normal', activation='selu'),
        BatchNormalization(),
        Dense(n_labels, kernel_initializer='lecun_normal', activation='selu'),
        BatchNormalization(),
        Dense(n_labels, activation="sigmoid")
        ],name='Head3')

    output = head_3(input_4_avg)


    model = Model(inputs = [input_1, input_2], outputs = output)
    model.compile(optimizer='adam', loss= BinaryCrossentropy(label_smoothing=label_smoothing), metrics=logloss)
    
    return model

num_all_ft = X_train.shape[1]
num_ft_1 = len(cols_gene)+640
num_ft_2 = num_all_ft - num_ft_1

model = build_model(num_ft_1, num_ft_2, len(cols_target))

In [58]:
num_ft_2

175

In [59]:
# Training and evaluate
reduce_lr = ReduceLROnPlateau(monitor='val_logloss', factor=0.3, patience=5, mode='min', min_lr=1E-5, verbose=1)
early_stopping = EarlyStopping(monitor='val_logloss', min_delta=1E-5, patience=15, mode='min',restore_best_weights=True, verbose=1)
# reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.3, patience=3,epsilon = 1e-4, mode = 'min',verbose=1)
# early_stopping = EarlyStopping(monitor='val_loss',min_delta=0,patience=10,mode='auto',verbose=1,baseline=None,restore_best_weights=True)

hist = model.fit([X_train[:,:num_ft_1], X_train[:,num_ft_1:]],y_train, batch_size=64, epochs=150,validation_split = 0.2
                 ,callbacks=[reduce_lr, early_stopping]
                )

Epoch 1/150
275/275 [==============================] - 3s 10ms/step - loss: 0.2116 - logloss: 0.2105 - val_loss: 0.0248 - val_logloss: 0.0236
Epoch 2/150
275/275 [==============================] - 2s 9ms/step - loss: 0.0216 - logloss: 0.0202 - val_loss: 0.0203 - val_logloss: 0.0188
Epoch 3/150
275/275 [==============================] - 3s 12ms/step - loss: 0.0191 - logloss: 0.0177 - val_loss: 0.0192 - val_logloss: 0.0177
Epoch 4/150
275/275 [==============================] - 3s 11ms/step - loss: 0.0180 - logloss: 0.0165 - val_loss: 0.0191 - val_logloss: 0.0176
Epoch 5/150
275/275 [==============================] - 3s 10ms/step - loss: 0.0171 - logloss: 0.0156 - val_loss: 0.0190 - val_logloss: 0.0174
Epoch 6/150
275/275 [==============================] - 3s 9ms/step - loss: 0.0162 - logloss: 0.0147 - val_loss: 0.0190 - val_logloss: 0.0173
Epoch 7/150
275/275 [==============================] - 2s 9ms/step - loss: 0.0152 - logloss: 0.0137 - val_loss: 0.0193 - val_logloss: 0.0176
Epoch 8/1

## **Hoàn thành dev model với tên biến model => submit**

In [60]:
# predict non ctl vehicle
X_pred = pipe.transform( to_pred_non_ctl[cols_fts] )
arr_preds_non_ctl = model.predict( [X_pred[:,:num_ft_1], X_pred[:,num_ft_1:]] )
#     model.predict(pipe.transform(to_pred_non_ctl[cols_fts]))


df_preds_non_ctl =  pd.DataFrame(arr_preds_non_ctl, columns= cols_target, index = to_pred_non_ctl.index)

# concat with all to pred values
df_preds = pd.concat([ to_pred[cols_id], df_preds_non_ctl], axis = 1).fillna(0)
# to csv
df_preds.to_csv("submission.csv", index = None)